<a href="https://colab.research.google.com/github/elangbijak4/Riset-AI-Medis/blob/main/Training_Model_3_Deteksi_Foto_X_Ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dan autentikasi
!pip install -q kaggle pydicom tqdm imgaug

from google.colab import drive
drive.mount('/content/drive')

# Unggah kaggle.json dari Google Drive ke folder ~/.kaggle
import os, io
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')
file_id = drive_service.files().list(q="name='kaggle.json'", fields="files(id)").execute()['files'][0]['id']
fh = io.FileIO('/root/.kaggle/kaggle.json', 'wb')
drive_service.files().get_media(fileId=file_id).download(fh)
os.chmod('/root/.kaggle/kaggle.json', 0o600)

# Download RSNA dataset (Stage 1)
!kaggle competitions download -c rsna-pneumonia-detection-challenge
!unzip -q stage_1_train_images.zip
!unzip -q stage_1_train_labels.csv.zip

# Alternatif: chestX-ray14 via Activeloop Deep Lake (jika tersedia)
!pip install -q deeplake
import deeplake
ds = deeplake.load('hub://activeloop/nih-chest-xray-train')

In [2]:
import pandas as pd, glob, pydicom
from imgaug import augmenters as iaa

# Load label CSV
df = pd.read_csv('stage_1_train_labels.csv')
# Contoh filter untuk pneumonia positif only
df_pos = df[df['Target'] == 1]

# Load dan augment gambar
def load_and_augment(fp):
    d = pydicom.dcmread(fp).pixel_array
    d = cv2.resize(d, (512,512))
    seq = iaa.Sequential([iaa.GaussianBlur((0,3.0))])
    return seq(image=d)

# Contoh pemrosesan per-batch
for idx, row in df_pos.sample(4).iterrows():
    img = load_and_augment(f"stage_1_train_images/{row['patientId']}.dcm")
    x1, y1, w, h = row[['x','y','width','height']]
    cv2.rectangle(img, (int(x1), int(y1)), (int(x1+w), int(y1+h)), (255,0,0), 2)
    plt.imshow(img, cmap='gray'); plt.show()

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Gunakan ImageDataGenerator untuk ChestX-ray14 ini
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_gen = datagen.flow_from_directory('/content/drive/MyDrive/datasets/ChestXray14',
                                         target_size=(224,224), class_mode='binary', subset='training')
val_gen   = datagen.flow_from_directory(..., subset='validation')

# Bangun model transfer learning
def build_model(base):
    base.trainable = False
    x = GlobalAveragePooling2D()(base.output)
    x = Dense(64, activation='relu')(x)
    out = Dense(1, activation='sigmoid')(x)
    m = Model(base.input, out)
    m.compile('adam', 'binary_crossentropy', ['accuracy'])
    return m

base = tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))
model = build_model(base)

# Fine-tune
history = model.fit(train_gen, validation_data=val_gen, epochs=5)